# VaR Estimation and Backtesting

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from IPython.display import display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
snp =pd.read_excel('\Data\S&P_BSE.xlsx') 

current_asset_value_snp = 1
units_snp = 1/snp['Close'].iloc[-1]

FileNotFoundError: [Errno 2] No such file or directory: '\\Data\\S&P_BSE.xlsx'

In [217]:
nif = pd.read_excel('\Data\NIFTY.xlsx') 


current_asset_value_nif = 1
units_nif = 1/nif['Close'].iloc[-1]

In [218]:
print("Current value of S&P BSE 500 as on",snp['Date'].iloc[-1],"is",current_asset_value_snp)
print("Current value of NIFTY as on",nif['Date'].iloc[-1],"is",current_asset_value_nif)


Current value of S&P BSE 500 as on 2021-04-09 00:00:00 is 1
Current value of NIFTY as on 2021-04-09 00:00:00 is 1


In [219]:
snp = (snp.set_index('Date'))
nif = (nif.set_index('Date'))

In [220]:
snp = snp['Close']
nif = nif['Close']

In [221]:
df = pd.DataFrame(columns=['VaR Terminology','S&P BSE 500','NIFTY'])

In [222]:
returns_snp = np.log(snp) - np.log(snp.shift(1))
returns_nif = np.log(nif) - np.log(nif.shift(1))

returns_snp = returns_snp.sort_values(ascending = True)
returns_nif = returns_nif.sort_values(ascending = True)

confidence_interval = 0.99
VaR_snp = returns_snp[int(len(returns_snp) * (1 - confidence_interval))] 
VaR_nif = returns_nif[int(len(returns_nif) * (1 - confidence_interval))] 

print("VaR for S&P returns is", round(VaR_snp*100,3), "%.")
print("VaR for Nifty returns is", round(VaR_nif*100,3), "%.")


VaR for S&P returns is -2.996 %.
VaR for Nifty returns is -2.977 %.


In [223]:
#df = df.append({'VaR Terminology':'% VaR','S&P BSE 500':-VaR_snp*100,'NIFTY':-VaR_nif*100}, ignore_index=True)

In [224]:
###### SNP #####
print("----------------------------- SnP stats -----------------------------")

# Emperical VaR
emperical_relative_VaR_snp = round(- current_asset_value_snp * VaR_snp,4)
print("Worst loss (Emperical Relative Loss) the asset value will face is Rs.", emperical_relative_VaR_snp)
emperical_absolute_VaR_snp = round(current_asset_value_snp * (np.mean(returns_snp) - VaR_snp),4)
print("Worst loss (Emperical Absolute Loss) the asset value will face is Rs.", emperical_absolute_VaR_snp)

# Parametric VaR
# Assumption: Time Series Data follows Normal Distribution
z = st.norm.ppf(confidence_interval)

parametric_relative_VaR_snp = round(current_asset_value_snp * z * np.std(returns_snp),4)
print("Worst loss (Parametric Relative Loss) the asset value will face is Rs.", parametric_relative_VaR_snp)
parametric_absolute_VaR_snp = round(- current_asset_value_snp * (np.mean(returns_snp) - z * np.std(returns_snp)),3)
print("Worst loss (Parametric Absolute Loss) the asset value will face is Rs.", parametric_absolute_VaR_snp)


print("----------------------------- Nifty stats -----------------------------")

# Emperical VaR
emperical_relative_VaR_nif = round(- current_asset_value_nif * VaR_nif,4) 
print("Worst loss (Emperical Relative Loss) the asset value will face is Rs.", emperical_relative_VaR_nif)
emperical_absolute_VaR_nif = round(current_asset_value_nif * (np.mean(returns_nif) - VaR_nif),4)
print("Worst loss (Emperical Absolute Loss) the asset value will face is Rs.", emperical_absolute_VaR_nif)

# Parametric VaR
# Assumption: Time Series Data follows Normal Distribution
z = st.norm.ppf(confidence_interval)

parametric_relative_VaR_nif = round(current_asset_value_nif * z * np.std(returns_nif),4)
print("Worst loss (Parametric Relative Loss) the asset value will face is Rs.", parametric_relative_VaR_nif)
parametric_absolute_VaR_nif = round(- current_asset_value_nif * (np.mean(returns_nif) - z * np.std(returns_nif)),4)
print("Worst loss (Parametric Absolute Loss) the asset value will face is Rs.", parametric_absolute_VaR_nif)


df = df.append({'VaR Terminology':'Emperical Absolute VaR','S&P BSE 500':emperical_absolute_VaR_snp,'NIFTY':emperical_absolute_VaR_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'Emperical Relative VaR','S&P BSE 500':emperical_relative_VaR_snp,'NIFTY':emperical_relative_VaR_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'Parametric Absolute VaR','S&P BSE 500':parametric_absolute_VaR_snp,'NIFTY':parametric_absolute_VaR_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'Parametric Relative VaR','S&P BSE 500':parametric_relative_VaR_snp,'NIFTY':parametric_relative_VaR_nif}, ignore_index=True)

----------------------------- SnP stats -----------------------------
Worst loss (Emperical Relative Loss) the asset value will face is Rs. 0.03
Worst loss (Emperical Absolute Loss) the asset value will face is Rs. 0.0306
Worst loss (Parametric Relative Loss) the asset value will face is Rs. 0.0269
Worst loss (Parametric Absolute Loss) the asset value will face is Rs. 0.026
----------------------------- Nifty stats -----------------------------
Worst loss (Emperical Relative Loss) the asset value will face is Rs. 0.0298
Worst loss (Emperical Absolute Loss) the asset value will face is Rs. 0.0303
Worst loss (Parametric Relative Loss) the asset value will face is Rs. 0.028
Worst loss (Parametric Absolute Loss) the asset value will face is Rs. 0.0274


In [225]:
# T - Day Returns : T = 7
# Considering efficient markets and log returns
print("----------------------------- SnP stats -----------------------------")
T = 7
std_T = np.std(returns_snp) * (T**0.5)
mean_T = np.mean(returns_snp) * T

confidence_interval = 0.99
z = st.norm.ppf(confidence_interval)

parametric_relative_T_VaR_snp = round(current_asset_value_snp * z * std_T,4)
print("Worst {} day loss (Parametric Relative Loss) the asset value will face is Rs. {}".format(T, parametric_relative_T_VaR_snp))
parametric_absolute_T_VaR_snp = round(- current_asset_value_snp * (mean_T - z * std_T),4)
print("Worst {} day loss (Parametric Absolute Loss) the asset value will face is Rs. {}".format(T, parametric_absolute_T_VaR_snp))


print("----------------------------- Nifty stats -----------------------------")
T = 7
std_T = np.std(returns_nif) * (T**0.5)
mean_T = np.mean(returns_nif) * T

confidence_interval = 0.99
z = st.norm.ppf(confidence_interval)

parametric_relative_T_VaR_nif = round(current_asset_value_nif * z * std_T,4)
print("Worst {} day loss (Parametric Relative Loss) the asset value will face is Rs. {}".format(T, parametric_relative_T_VaR_nif))
parametric_absolute_T_VaR_nif = round(- current_asset_value_nif * (mean_T - z * std_T),4)
print("Worst {} day loss (Parametric Absolute Loss) the asset value will face is Rs. {}".format(T, parametric_absolute_T_VaR_nif))


df = df.append({'VaR Terminology':'7 day loss-absolute (Efficient markets)','S&P BSE 500':parametric_absolute_T_VaR_snp,'NIFTY':parametric_absolute_T_VaR_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'7 day loss-relative (Efficient markets)','S&P BSE 500':parametric_relative_T_VaR_snp,'NIFTY':parametric_relative_T_VaR_nif}, ignore_index=True)

----------------------------- SnP stats -----------------------------
Worst 7 day loss (Parametric Relative Loss) the asset value will face is Rs. 0.0712
Worst 7 day loss (Parametric Absolute Loss) the asset value will face is Rs. 0.0669
----------------------------- Nifty stats -----------------------------
Worst 7 day loss (Parametric Relative Loss) the asset value will face is Rs. 0.074
Worst 7 day loss (Parametric Absolute Loss) the asset value will face is Rs. 0.07


In [226]:
# T - Day Returns : T = 7
# Considering inefficient markets and log returns
print("----------------------------- SnP stats -----------------------------")
T = 7
covariance_return = 0.05
std_T = (T * (np.std(returns_snp)**2) + 2 * (np.std(returns_snp)**2) * sum([((T - i) * (covariance_return**i)) for i in range(1, T)]))**0.5 
mean_T = np.mean(returns_snp) * T

confidence_interval = 0.99
z = st.norm.ppf(confidence_interval)

parametric_relative_T_VaR_snp = round(current_asset_value_snp * z * std_T,4)
print("Worst {} day loss (Parametric Relative Loss) the asset value will face is Rs. {}".format(T, parametric_relative_T_VaR_snp))
parametric_absolute_T_VaR_snp = round(- current_asset_value_snp * (mean_T - z * std_T),4)
print("Worst {} day loss (Parametric Absolute Loss) the asset value will face is Rs. {}".format(T, parametric_absolute_T_VaR_snp))


print("----------------------------- Nifty stats -----------------------------")
T = 7
covariance_return = 0.05
std_T = (T * (np.std(returns_nif)**2) + 2 * (np.std(returns_nif)**2) * sum([((T - i) * (covariance_return**i)) for i in range(1, T)]))**0.5 
mean_T = np.mean(returns_nif) * T

confidence_interval = 0.99
z = st.norm.ppf(confidence_interval)

parametric_relative_T_VaR_nif = round(current_asset_value_nif * z * std_T,4)
print("Worst {} day loss (Parametric Relative Loss) the asset value will face is Rs. {}".format(T, parametric_relative_T_VaR_nif))
parametric_absolute_T_VaR_nif = round(- current_asset_value_nif * (mean_T - z * std_T),4)
print("Worst {} day loss (Parametric Absolute Loss) the asset value will face is Rs. {}".format(T, parametric_absolute_T_VaR_nif))


df = df.append({'VaR Terminology':'7 day loss-absolute (Inefficient markets)','S&P BSE 500':parametric_absolute_T_VaR_snp,'NIFTY':parametric_absolute_T_VaR_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'7 day loss-relative (Inefficient markets)','S&P BSE 500':parametric_relative_T_VaR_snp,'NIFTY':parametric_relative_T_VaR_nif}, ignore_index=True)

----------------------------- SnP stats -----------------------------
Worst 7 day loss (Parametric Relative Loss) the asset value will face is Rs. 0.0743
Worst 7 day loss (Parametric Absolute Loss) the asset value will face is Rs. 0.07
----------------------------- Nifty stats -----------------------------
Worst 7 day loss (Parametric Relative Loss) the asset value will face is Rs. 0.0772
Worst 7 day loss (Parametric Absolute Loss) the asset value will face is Rs. 0.0732


In [227]:
# cVaR
confidence_interval = 0.99
Z_CI = -st.norm.ppf(1 - confidence_interval)
Z_cVaR = np.exp((-(Z_CI**2)/2))/((1 - confidence_interval)*np.sqrt(2*3.14))

print("----------------------------- SnP stats -----------------------------")
cVaR_snp = round(-Z_cVaR*np.std(returns_snp),4) 
print("cVaR :", cVaR_snp*100, "%")

# cVaR
relative_cVaR_snp = round(current_asset_value_snp * (np.mean(returns_snp) - cVaR_snp),4)
print("Worst loss (Relative Loss) the asset value will face is Rs.", relative_cVaR_snp)
absolute_cVaR_snp = round(- current_asset_value_snp * cVaR_snp,4)
print("Worst loss (Absolute Loss) the asset value will face is Rs.", absolute_cVaR_snp)


print("----------------------------- Nifty stats -----------------------------")
cVaR_nif = round(-Z_cVaR*np.std(returns_nif),4)
print("cVaR :", cVaR_nif*100, "%")

# cVaR
relative_cVaR_nif = round(current_asset_value_nif * (np.mean(returns_nif) - cVaR_nif),4)
print("Worst loss (Relative Loss) the asset value will face is Rs.", relative_cVaR_nif)
absolute_cVaR_nif = round(- current_asset_value_nif * cVaR_nif,4)
print("Worst loss (Absolute Loss) the asset value will face is Rs.", absolute_cVaR_nif)


df = df.append({'VaR Terminology':'% CVaR','S&P BSE 500':-cVaR_snp*100,'NIFTY':-cVaR_nif*100}, ignore_index=True)
df = df.append({'VaR Terminology':'Worst loss (Absolute Loss)','S&P BSE 500':absolute_cVaR_snp,'NIFTY':absolute_cVaR_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'Worst loss (Relative Loss)','S&P BSE 500':relative_cVaR_snp,'NIFTY':relative_cVaR_nif}, ignore_index=True)

----------------------------- SnP stats -----------------------------
cVaR : -3.08 %
Worst loss (Relative Loss) the asset value will face is Rs. 0.0314
Worst loss (Absolute Loss) the asset value will face is Rs. 0.0308
----------------------------- Nifty stats -----------------------------
cVaR : -3.2 %
Worst loss (Relative Loss) the asset value will face is Rs. 0.0326
Worst loss (Absolute Loss) the asset value will face is Rs. 0.032


In [228]:
df['Relative (Nifty/ S&P)'] = df['NIFTY']/df['S&P BSE 500']
df['Remarks'] = 0

In [229]:
for i in range(df.shape[0]): 
    if df['Relative (Nifty/ S&P)'][i]>=1:
        df['Remarks'][i] = "NIFTY is riskier"
    else:
        df['Remarks'][i] = "S&P is riskier"

In [230]:
df

,VaR Terminology,S&P BSE 500,NIFTY,Relative (Nifty/ S&P),Remarks
0,Emperical Absolute VaR,0.0306,0.0303,0.990196,S&P is riskier
1,Emperical Relative VaR,0.0300,0.0298,0.993333,S&P is riskier
2,Parametric Absolute VaR,0.0260,0.0274,1.053846,NIFTY is riskier
3,Parametric Relative VaR,0.0269,0.0280,1.040892,NIFTY is riskier
4,7 day loss-absolute (Efficient markets),0.0669,0.0700,1.046338,NIFTY is riskier
5,7 day loss-relative (Efficient markets),0.0712,0.0740,1.039326,NIFTY is riskier
6,7 day loss-absolute (Inefficient markets),0.0700,0.0732,1.045714,NIFTY is riskier
7,7 day loss-relative (Inefficient markets),0.0743,0.0772,1.039031,NIFTY is riskier
8,% CVaR,3.0800,3.2000,1.038961,NIFTY is riskier
9,Worst loss (Absolute Loss),0.0308,0.0320,1.038961,NIFTY is riskier


In [231]:
df.to_csv (r'/Users/nesarasr/Documents/Sem6/FRM/Termproject/Results_VaREstimates.csv',index = False,  header=True)

In [232]:
# Ranged VaR 
print("----------------------------- SnP stats -----------------------------")
range_confidence_interval = 0.95
VaR_confidence_interval = 0.99
# We need to calculate 2 tailed critical value for z_range
z_range = st.norm.ppf(range_confidence_interval + (1 - range_confidence_interval)/2)
z_VaR = st.norm.ppf(VaR_confidence_interval)

print("Sample Standard Deviation :", np.std(returns_snp))
std_min = np.std(returns_snp) - z_range * np.std(returns_snp) / ((2 * len(returns_snp))**0.5)
print("Mininum Sample Standard Deviation (from standard error):", std_min)
std_max = np.std(returns_snp) + z_range * np.std(returns_snp) / ((2 * len(returns_snp))**0.5)
print("Maximum Sample Standard Deviation (from standard error):", std_max)

print("Sample Mean :", np.mean(returns_snp))
mean_min = np.mean(returns_snp) - z_range * np.std(returns_snp) / (len(returns_snp)**0.5)
print("Minimum Mean (from standard error):", mean_min)
mean_max = np.mean(returns_snp) + z_range * np.std(returns_snp) / (len(returns_snp)**0.5)
print("Maximum Mean (from standard error):", mean_max)

parametric_relative_VaR_min = current_asset_value_snp * z_VaR * std_min
parametric_relative_VaR_max = current_asset_value_snp * z_VaR * std_max
print("Min Parametric Relative Loss the asset value will face is Rs.", parametric_relative_VaR_min)
print("Max Parametric Relative Loss the asset value will face is Rs.", parametric_relative_VaR_max)
parametric_absolute_VaR_min = - current_asset_value_snp * (mean_max - z_VaR * std_min)
parametric_absolute_VaR_max = - current_asset_value_snp * (mean_min - z_VaR * std_max)
#parametric_absolute_VaR_x = - current_asset_value * (mean_min - z_VaR * std_min)
#parametric_absolute_VaR_y = - current_asset_value * (mean_max - z_VaR * std_max)
#print(parametric_absolute_VaR_min, parametric_absolute_VaR_max, parametric_absolute_VaR_x, parametric_absolute_VaR_y)
print("Min Parametric Absolute Loss the asset value will face is Rs.", parametric_absolute_VaR_min)
print("Max Parametric Absolute Loss the asset value will face is Rs.", parametric_absolute_VaR_max)

print("----------------------------- Nifty stats -----------------------------")
range_confidence_interval = 0.95
VaR_confidence_interval = 0.99
# We need to calculate 2 tailed critical value for z_range
z_range = st.norm.ppf(range_confidence_interval + (1 - range_confidence_interval)/2)
z_VaR = st.norm.ppf(VaR_confidence_interval)

print("Sample Standard Deviation :", np.std(returns_nif))
std_min = np.std(returns_nif) - z_range * np.std(returns_nif) / ((2 * len(returns_nif))**0.5)
print("Mininum Sample Standard Deviation (from standard error):", std_min)
std_max = np.std(returns_nif) + z_range * np.std(returns_nif) / ((2 * len(returns_nif))**0.5)
print("Maximum Sample Standard Deviation (from standard error):", std_max)

print("Sample Mean :", np.mean(returns_nif))
mean_min = np.mean(returns_nif) - z_range * np.std(returns_nif) / (len(returns_nif)**0.5)
print("Minimum Mean (from standard error):", mean_min)
mean_max = np.mean(returns_nif) + z_range * np.std(returns_nif) / (len(returns_nif)**0.5)
print("Maximum Mean (from standard error):", mean_max)

parametric_relative_VaR_min = current_asset_value_nif * z_VaR * std_min
parametric_relative_VaR_max = current_asset_value_nif * z_VaR * std_max
print("Min Parametric Relative Loss the asset value will face is Rs.", parametric_relative_VaR_min)
print("Max Parametric Relative Loss the asset value will face is Rs.", parametric_relative_VaR_max)
parametric_absolute_VaR_min = - current_asset_value_nif * (mean_max - z_VaR * std_min)
parametric_absolute_VaR_max = - current_asset_value_nif * (mean_min - z_VaR * std_max)
#parametric_absolute_VaR_x = - current_asset_value * (mean_min - z_VaR * std_min)
#parametric_absolute_VaR_y = - current_asset_value * (mean_max - z_VaR * std_max)
#print(parametric_absolute_VaR_min, parametric_absolute_VaR_max, parametric_absolute_VaR_x, parametric_absolute_VaR_y)
print("Min Parametric Absolute Loss the asset value will face is Rs.", parametric_absolute_VaR_min)
print("Max Parametric Absolute Loss the asset value will face is Rs.", parametric_absolute_VaR_max)

----------------------------- SnP stats -----------------------------
Sample Standard Deviation : 0.011570489305660563
Mininum Sample Standard Deviation (from standard error): 0.011277769693714298
Maximum Sample Standard Deviation (from standard error): 0.011863208917606827
Sample Mean : 0.0006163328110222614
Minimum Mean (from standard error): 0.00020236476583526366
Maximum Mean (from standard error): 0.0010303008562092591
Min Parametric Relative Loss the asset value will face is Rs. 0.02623601555089448
Max Parametric Relative Loss the asset value will face is Rs. 0.027597950844776988
Min Parametric Absolute Loss the asset value will face is Rs. 0.025205714694685223
Max Parametric Absolute Loss the asset value will face is Rs. 0.027395586078941724
----------------------------- Nifty stats -----------------------------
Sample Standard Deviation : 0.012021814408682599
Mininum Sample Standard Deviation (from standard error): 0.011717676808652322
Maximum Sample Standard Deviation (from st

### Backtesting

In [233]:
# Conditional Coverage BackTesting
snp =pd.read_excel('/Users/nesarasr/Documents/Sem6/FRM/Termproject/S&P_BSE.xlsx') 
snp = snp.Close.to_numpy()
snp = snp[-201:]

nif = pd.read_excel('/Users/nesarasr/Documents/Sem6/FRM/Termproject/NIFTY.xlsx') 
nif = nif.Close.to_numpy()
nif = nif[-201:]

print("----------------------------- SnP stats -----------------------------")
VaR = 0.026900
loss = []
for i in range(snp.shape[0]):
    if i == 0:
        loss.append(0)
    else:
        loss.append(min(0, units_snp*(snp[i] - snp[i - 1])))
        
christofferson_values = []
for i in range(snp.shape[0]):
    if round(abs(loss[i]),4) > VaR:
        christofferson_values.append(1)
    else:
        christofferson_values.append(0)

N = 0
for i in range(1, snp.shape[0]):
    if christofferson_values[i] == 1:
        N += 1
T = len(christofferson_values) - 1
print("(N, T) = ", N, T)

one_sided_confidence_interval = 0.99
confidence_interval = 1 - (1 - one_sided_confidence_interval) / 2
Zcalc = (N - T * (1 - confidence_interval)) / (T * confidence_interval * (1 - confidence_interval)) ** 0.5
Ztab = st.norm.ppf(confidence_interval)
print("(Zcalc, Ztab) = ", Zcalc, Ztab)

if Zcalc < Ztab and Zcalc > -Ztab:
    print("Model is GOOD as per conditional coverage test")
else:
    print("Model is NOT GOOD as per conditional coverage test")



----------------------------- SnP stats -----------------------------
(N, T) =  2 200
(Zcalc, Ztab) =  1.0025094142341697 2.5758293035489004
Model is GOOD as per conditional coverage test


In [234]:
print("----------------------------- Nifty stats -----------------------------")
VaR = 0.028000
loss = []
for i in range(nif.shape[0]):
    if i == 0:
        loss.append(0)
    else:
        loss.append(min(0, units_nif*(nif[i] - nif[i - 1])))
        
christofferson_values = []
for i in range(nif.shape[0]):
    if round(abs(loss[i]),4) > VaR:
        christofferson_values.append(1)
    else:
        christofferson_values.append(0)

N = 0
for i in range(1, nif.shape[0]):
    if christofferson_values[i] == 1:
        N += 1
T = len(christofferson_values) - 1
print("(N, T) = ", N, T)

one_sided_confidence_interval = 0.99
confidence_interval = 1 - (1 - one_sided_confidence_interval) / 2
Zcalc = (N - T * (1 - confidence_interval)) / (T * confidence_interval * (1 - confidence_interval)) ** 0.5
Ztab = st.norm.ppf(confidence_interval)
print("(Zcalc, Ztab) = ", Zcalc, Ztab)

if Zcalc < Ztab and Zcalc > -Ztab:
    print("Model is GOOD as per conditional coverage test")
else:
    print("Model is NOT GOOD as per conditional coverage test")

----------------------------- Nifty stats -----------------------------
(N, T) =  2 200
(Zcalc, Ztab) =  1.0025094142341697 2.5758293035489004
Model is GOOD as per conditional coverage test
